In [4]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd
import keras as K

In [16]:
def choose_log_file():
    data = list()
    filename = f'./1 - Logs Here/test.log.1'
    file = open(filename, 'r')
    lines = file.read().splitlines()
    file.close()
    for line in lines:
        if not line:
            continue
        substring = '"-"'
        columns2 = [col.strip() for col in line.split('"-"') if col]
        while substring in columns2:
            columns2 = [col.strip() for col in columns2.split('"-"') if col]
        impData = columns2[0]
        impData2 = [col.strip() for col in impData.split(' - - ')]
        try:
            testString = impData2[1].split('"')
        except:
            continue
        date = testString[0].strip(' ')
        request = testString[1]
        requestResponseCode = testString[2].split(' ')

        try:
            requestResponseCodeShortened = requestResponseCode[1]
        except:
            requestResponseCodeShortened = np.nan

        if requestResponseCodeShortened == '200' or requestResponseCodeShortened == '201' or requestResponseCodeShortened == '202':
            columnToAdd = '0'
        else:
            columnToAdd = '1'
        data.append([impData2[0],date,request,requestResponseCodeShortened,columnToAdd])
    print('Log successfully parsed!')

    df = pd.DataFrame(data=data)
    df.columns = ['IP', 'Datetime', 'Request', 'StatusCode', 'Target']
    df = df.dropna()
    return np.array(data), df

def split_dataset(df):
    x = df.values[:,0:4]
    y = df.values[:,4]
    df.values[:,3:5] = df.values[:,3:5].astype(str)

    #encode the x dataset
    # Get unique classes (encuentra los elementos unicos en el array).
    x_dataset_base = x
    y_dataset_base = y
    x_dataset_ips = x_dataset_base[:,0]
    x_dataset_dates = x_dataset_base[:,1]
    uniqueValues = np.unique(x_dataset_ips)
    classes = np.unique(x)
    #for value in x_dataset_ips:

    # Replace classes with integers.
    x = np.searchsorted(classes, x)
    inputColumnQuantityVariables = len(x[0])

    #encode the y dataset
    y = np.array(y.astype(int))

    return x, y, x_dataset_ips, x_dataset_dates

def predict_attacks(x_dataset_ips):
    predictions = model.predict(x)

    possible_attack_hosts = []
    done_values = []
    bad_requests = 0

    for i, element in enumerate(x_dataset_ips):
        previous_element = x_dataset_ips[i - 1] if i > 0 else None
        current_element = element
        if current_element != previous_element and current_element not in done_values:
            done_values.append(current_element)
            times_appeared = np.count_nonzero(x_dataset_ips == current_element)
            lines_in = np.array(np.where(x_dataset_ips == current_element))
            lines_in_added = np.array([a+1 for a in lines_in])

            possible_attack_hosts.append((current_element,times_appeared))
            if times_appeared > 100:
    #             possible_attack_hosts.append([current_element,times_appeared])
                bad_request_dates = []
                for line_number in lines_in:
                    for numb in line_number:
                        if predictions[numb] == 1:
                            #bad_request_lines = array(numpy.where(predictions == predictions[numb]))
                            bad_requests = bad_requests + 1
                        #for date in x_dataset_dates:
                            bad_request_dates.append(x_dataset_dates[numb])
            bad_requests = 0
    return possible_attack_hosts


In [17]:
dataset, df = choose_log_file()
x, y, x_dataset_ips, x_dataset_dates = split_dataset(df)
model = K.models.load_model('./model.h5')
possible_attack_hosts = predict_attacks(x_dataset_ips)

Log successfully parsed!


In [23]:
for i, attacker in enumerate(possible_attack_hosts):
    if attacker[1] <= 15:
        attacker_categorized = (attacker[0], attacker[1], 0)
        possible_attack_hosts[i] = attacker_categorized
        continue
    if attacker[1] <= 50:
        attacker_categorized = (attacker[0], attacker[1], 1)
        possible_attack_hosts[i] = attacker_categorized
        continue
    if attacker[1] <= 200:
        attacker_categorized = (attacker[0], attacker[1], 2)
        possible_attack_hosts[i] = attacker_categorized
        continue
    if attacker[1] <= 500:
        attacker_categorized = (attacker[0], attacker[1], 3)
        possible_attack_hosts[i] = attacker_categorized
        continue
    
    attacker_categorized = (attacker[0], attacker[1], 4)
    possible_attack_hosts[i] = attacker_categorized

In [24]:
possible_attack_hosts

[('208.80.193.39', 16, 1),
 ('82.80.249.160', 1, 0),
 ('198.186.192.44', 6, 0),
 ('66.249.71.179', 2, 0),
 ('208.80.193.29', 2, 0),
 ('66.249.68.210', 2, 0),
 ('184.73.2.36', 2, 0),
 ('208.80.193.42', 3, 0),
 ('208.80.193.43', 2, 0),
 ('188.140.124.209', 3, 0),
 ('208.80.193.30', 3, 0),
 ('207.44.204.87', 14, 0),
 ('208.80.193.37', 2, 0),
 ('89.180.153.31', 3, 0),
 ('67.195.111.233', 16, 1),
 ('217.129.199.36', 3, 0),
 ('208.80.193.35', 1, 0),
 ('174.129.95.108', 1, 0),
 ('66.249.65.53', 2, 0),
 ('208.80.193.40', 5, 0),
 ('87.103.59.236', 1, 0),
 ('184.73.90.204', 1, 0),
 ('164.143.244.33', 2, 0),
 ('82.155.172.156', 2, 0),
 ('66.249.65.124', 2, 0),
 ('81.35.154.75', 1, 0),
 ('64.246.161.30', 6, 0),
 ('72.44.53.231', 1, 0),
 ('69.58.178.27', 2, 0),
 ('208.80.193.34', 3, 0),
 ('65.52.108.59', 2, 0),
 ('174.129.178.134', 2, 0),
 ('66.249.65.252', 2, 0),
 ('66.249.65.226', 2, 0),
 ('208.80.193.27', 4, 0),
 ('184.73.87.56', 2, 0),
 ('208.80.193.121', 6, 0),
 ('184.73.61.52', 2, 0),
 ('207.